## Cat Classifier
Determines whether an image contains a cat<br>
Dataset taken from Andrew Ng's Coursera course<br>
Last update: 12/30/23

In [1]:
import cupy as np # Use Cuda
from h5py import File
from importlib import reload
from PIL import Image
from sandbox import activations, costs, initializers, layers, model, optimizers, utils

In [2]:
# Load data
train_dataset = File('dataset\\train_catvnoncat.h5', 'r')
train_x = np.array(train_dataset['train_set_x'][:]) # Train set features
train_y = np.array(train_dataset['train_set_y'][:]) # Train set labels

test_dataset = File('dataset\\test_catvnoncat.h5', 'r')
test_x = np.array(test_dataset['test_set_x'][:]) # Test set features
test_y = np.array(test_dataset['test_set_y'][:]) # Test set labels

train_y = train_y.reshape((train_y.shape[0], 1))
test_y = test_y.reshape((test_y.shape[0], 1))

# Flatten and normalize
train_x = train_x.reshape(train_x.shape[0], -1) / 255
test_x = test_x.reshape(test_x.shape[0], -1) / 255

In [22]:
reload(model)

# Create model
cat = model.Model(cuda=True)
cat.add(layers.Dense(units=20, activation=activations.ReLU()))
cat.add(layers.Dense(units=7, activation=activations.ReLU()))
cat.add(layers.Dense(units=5, activation=activations.ReLU()))
cat.add(layers.Dense(units=1, activation=activations.Sigmoid()))

cat.configure(
    input_size=train_x.shape[1],
    cost_type=costs.BinaryCrossentropy(),
    optimizer=optimizers.Adam()
)

# Train model
cat.train(train_x, train_y, epochs=80, batch_size=64, verbose=True)
cat.save() # Save model parameters

Cost on epoch 8: 0.51473
Cost on epoch 16: 0.48537
Cost on epoch 24: 0.40766
Cost on epoch 32: 0.33317
Cost on epoch 40: 0.32407
Cost on epoch 48: 0.37087
Cost on epoch 56: 0.1657
Cost on epoch 64: 0.18533
Cost on epoch 72: 0.13632
Cost on epoch 80: 0.22034


In [24]:
# Print model summary
cat.summary()

# Assess model accuracy
pred_train = cat.predict(train_x) # Get model accuracy on training data
print('\nTraining Accuracy:', utils.binary_evaluate(pred_train, train_y))
pred_test = cat.predict(test_x) # Get model accuracy on testing data
print('Testing Accuracy:', utils.binary_evaluate(pred_test, test_y))

# Use model on custom image
my_image = 'dataset\\cat.jpg' 
num_px = 64

image = np.array(Image.open(my_image).resize((num_px, num_px))) / 255 # Resize and normalize image, cast to NumPy array
image = image.reshape((1, num_px * num_px * 3)) # Flatten image array
my_predicted_image = utils.binary_round(cat.predict(image)) # Predict custom image
print(f"Image guess: {my_predicted_image.item()}")

+------------+------------+
| Layer type | Parameters |
+------------+------------+
|   Dense    |   245780   |
|   Dense    |    147     |
|   Dense    |     40     |
|   Dense    |     6      |
+------------+------------+
Total parameters: 245973

Training Accuracy: 0.9569377990430622
Testing Accuracy: 0.74
Image guess: 0
